In [1]:
import sys
import time

sys.path.append("../")
from calculus.pseudonym import *
from calculus.strategy import *

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = LeftmostOutermostStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._updateBoundVariables()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._betaConversion(reduction_strategy)
        term = term._updateBoundVariables()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


# def pred_term():
#     n = Var()
#     n_ = Atom(n)
#     return Lambda(
#         n,
#         App(
#             first_term(),
#             multi_app_term(
#                 n_, sinc_term(),
#                 multi_app_term(pair_term(), num_term(0), num_term(0))
#             ),
#         ),
#     )


# (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))
# (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))
def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λq.(λw.w)) ((λe.(e e)) (λr.(r r))))
1: (λq.q)


reduction steps: 1
norm term:       (λq.q)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λq.(λw.(λe.((q w) e)))) (λr.(λt.r))) (λy.(λu.(y (y (y u)))))) (λi.(λo.(i o))))
1: (((λw.(λe.(((λr.(λt.r)) w) e))) (λy.(λu.(y (y (y u)))))) (λi.(λq.(i q))))
2: ((λq.(((λw.(λe.w)) (λr.(λt.(r (r (r t)))))) q)) (λy.(λu.(y u))))
3: (((λq.(λw.q)) (λe.(λr.(e (e (e r)))))) (λt.(λy.(t y))))
4: ((λq.(λw.(λe.(w (w (w e)))))) (λr.(λt.(r t))))
5: (λq.(λw.(q (q (q w)))))


reduction steps: 5
norm term:       (λq.(λw.(q (q (q w)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λi.(λo.(λq.((i o) q)))) (λw.(λe.e))) (λr.(λt.t))) (λy.(λu.(y (y (y (y u)))))))
1: (((λq.(λw.(((λe.(λr.r)) q) w))) (λt.(λy.y))) (λu.(λi.(u (u (u (u i)))))))
2: ((λq.(((λw.(λe.e)) (λr.(λt.t))) q)) (λy.(λu.(y (y (y (y u)))))))
3: (((λq.(λw.w)) (λe.(λr.r))) (λt.(λy.(t (t (t (t y)))))))
4: ((λq.q) (λw.(λe.(w (w (w (w e)))))))
5: (λw.(λq.(w (w (w (w q))))))


reduction steps: 5
norm term:       (λw.(λq.(w (w (w (w q))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [18]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) (λi.(λo.o)))
1: (λq.(λw.((((λe.(λr.r)) (λt.(λy.(y (t q))))) (λu.w)) (λi.i))))
2: (λq.(λw.(((λe.e) (λr.w)) (λt.t))))
3: (λq.(λw.((λe.w) (λr.r))))
4: (λq.(λw.w))


reduction steps: 4
norm term:       (λq.(λw.w))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λt.(λy.(λu.(((t (λi.(λo.(o (i y))))) (λq.u)) (λw.w))))) (λe.(λr.(e (e (e (e (e r))))))))
1: (λq.(λw.((((λe.(λr.(e (e (e (e (e r))))))) (λt.(λy.(y (t q))))) (λu.w)) (λi.i))))
2: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) ((λs.(λd.(d (s q)))) e)))))) (λf.w)) (λg.g))))
3: (λp.(λa.(((λs.(λd.(d (s p)))) ((λf.(λq.(q (f p)))) ((λw.(λe.(e (w p)))) ((λr.(λt.(t (r p)))) ((λy.(λu.(u (y p)))) (λi.a)))))) (λo.o))))
4: (λq.(λw.((λe.(e (((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) (λs.w))))) q))) (λd.d))))
5: (λe.(λr.((λt.t) (((λy.(λu.(u (y e)))) ((λi.(λo.(o (i e)))) ((λp.(λa.(a (p e)))) ((λs.(λq.(q (s e)))) (λw.r))))) e))))
6: (λq.(λw.(((λe.(λr.(r (e q)))) ((λt.(λy.(y (t q)))) ((λu.(λi.(i (u q)))) ((λo.(λp.(p (o q)))) (λa.w))))) q)))
7: (λq.(λw.((λe.(e (((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) (λp.w)))) q))) q)))
8: (λq.(λw.(q (((λe.(λr.(r (e q)))) ((λt.(λy.(y (t q)))) ((λu.(λi.(i

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λq.(λw.((w (λe.(λr.(λt.(r ((e r) t)))))) q))) (λy.(λu.u))) (λi.(λo.o)))
1: ((λq.((q (λw.(λe.(λr.(e ((w e) r)))))) (λt.(λy.y)))) (λu.(λi.i)))
2: (((λt.(λy.y)) (λu.(λq.(λw.(q ((u q) w)))))) (λe.(λr.r)))
3: ((λq.q) (λw.(λe.e)))
4: (λw.(λq.q))


reduction steps: 4
norm term:       (λw.(λq.q))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λq.(λw.((w (λe.(λr.(λt.(r ((e r) t)))))) q))) (λy.(λu.(y (y u))))) (λi.(λo.(i (i (i o))))))
1: ((λr.((r (λt.(λy.(λu.(y ((t y) u)))))) (λi.(λq.(i (i q)))))) (λw.(λe.(w (w (w e))))))
2: (((λq.(λw.(q (q (q w))))) (λe.(λr.(λt.(r ((e r) t)))))) (λy.(λu.(y (y u)))))
3: ((λq.((λw.(λe.(λr.(e ((w e) r))))) ((λt.(λy.(λu.(y ((t y) u))))) ((λi.(λo.(λp.(o ((i o) p))))) q)))) (λa.(λs.(a (a s)))))
4: ((λw.(λe.(λr.(e ((w e) r))))) ((λt.(λy.(λu.(y ((t y) u))))) ((λi.(λo.(λp.(o ((i o) p))))) (λa.(λq.(a (a q)))))))
5: (λq.(λw.(q ((((λe.(λr.(λt.(r ((e r) t))))) ((λy.(λu.(λi.(u ((y u) i))))) (λo.(λp.(o (o p)))))) q) w))))
6: (λq.(λw.(q (((λe.(λr.(e ((((λt.(λy.(λu.(y ((t y) u))))) (λi.(λo.(i (i o))))) e) r)))) q) w))))
7: (λq.(λw.(q ((λe.(q ((((λr.(λt.(λy.(t ((r t) y))))) (λu.(λi.(u (u i))))) q) e))) w))))
8: (λr.(λt.(r (r ((((λy.(λu.(λq.(u ((y u) q))))) (λw.(λe.(w (w e))))) r) t)))))
9: (λq.(λw.(q (q (((λe.(λr.(e (((λt.(λy.(t (t y)))) e) r)))) q) w)))))
10: (λq.(λw.(q (q ((λe.(q (((λr.(λt.(r (r t))))

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λq.(λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q))) (λp.(λa.(p (p (p (p a))))))) (λs.(λd.(s d))))
1: ((λq.((q (λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i)))))) (λo.(λp.(o (o (o (o p)))))))) (λa.(λs.(a s))))
2: (((λi.(λo.(i o))) (λp.(λa.(λq.(((p (λw.(λe.(e (w a))))) (λr.q)) (λt.t)))))) (λy.(λu.(y (y (y (y u)))))))
3: ((λq.((λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i))))) q)) (λo.(λp.(o (o (o (o p)))))))
4: ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) (λi.(λo.(i (i (i (i o)))))))
5: (λe.(λr.((((λt.(λy.(t (t (t (t y)))))) (λu.(λi.(i (u e))))) (λq.r)) (λw.w))))
6: (λq.(λw.(((λe.((λr.(λt.(t (r q)))) ((λy.(λu.(u (y q)))) ((λi.(λo.(o (i q)))) ((λp.(λa.(a (p q)))) e))))) (λs.w)) (λd.d))))
7: (λq.(λw.(((λe.(λr.(r (e q)))) ((λt.(λy.(y (t q)))) ((λu.(λi.(i (u q)))) ((λo.(λp.(p (o q)))) (λa.w))))) (λs.s))))
8: (λu.(λi.((λo.(o (((λp.(λa.(a (p u)))) ((λq.(λw.(w (q u)))) ((λe.(λr.(r (e u)))) (λt.i)))) u))) (λy.y))))
9: (λq.(λw.((λe.e) (((λr.(λt

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9(), steps_lim=150)

0: (((λq.(λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q))) (λp.(λa.(p (p (p a)))))) (λs.(λd.(s (s (s (s (s d))))))))
1: ((λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p a))))))) (λs.(λq.(s (s (s (s (s q))))))))
2: (((λq.(λw.(q (q (q (q (q w))))))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p a))))))
3: ((λq.((λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i))))) ((λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g))))) ((λh.(λj.(λk.(((h (λl.(λz.(z (l j))))) (λx.k)) (λc.c))))) ((λv.(λb.(λn.(((v (λm.(λq_1.(q_1 (m b))))) (λw_1.n)) (λe_1.e_1))))) ((λr_1.(λt_1.(λy_1.(((r_1 (λu_1.(λi_1.(i_1 (u_1 t_1))))) (λo_1.y_1)) (λp_1.p_1))))) q)))))) (λa_1.(λs_1.(a_1 (a_1 (a_1 s_1))))))
4: ((λq.(λw.(λe.(((q (λr.(λt.(t (r w))))) (λy.e)) (λu.u))))) ((λi.(λo.(λp.(((i (λa.(λs.(s (a o))))) (λd.p)) (λf.f))))) ((λg.(λh.(λj.(((g (λk.(λl.(l (k h))))) (λz.j)) (λx.x))))) ((λc.(λv.(λb.(((c (λn.(λm.(m (n v))))) (λq_1.b)) (λw_1.w_

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λe.(λr.((λt.((t (λy.(λu.(λi.i)))) (λo.(λp.o)))) (((λa.(λs.((s (λd.(λf.(λg.(((d (λh.(λj.(j (h f))))) (λk.g)) (λl.l)))))) a))) e) r)))) (λz.(λx.(z (z (z x)))))) (λq.(λw.(q (q w)))))
1: ((λq.((λw.((w (λe.(λr.(λt.t)))) (λy.(λu.y)))) (((λi.(λo.((o (λp.(λa.(λs.(((p (λd.(λf.(f (d a))))) (λg.s)) (λh.h)))))) i))) (λj.(λk.(j (j (j k)))))) q))) (λl.(λz.(l (l z)))))
2: ((λq.((q (λw.(λe.(λr.r)))) (λt.(λy.t)))) (((λu.(λi.((i (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) u))) (λh.(λj.(h (h (h j)))))) (λk.(λl.(k (k l))))))
3: (((((λq.(λw.((w (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) q))) (λp.(λa.(p (p (p a)))))) (λs.(λd.(s (s d))))) (λf.(λg.(λh.h)))) (λj.(λk.j)))
4: ((((λd.((d (λf.(λg.(λh.(((f (λj.(λq.(q (j g))))) (λw.h)) (λe.e)))))) (λr.(λt.(r (r (r t))))))) (λy.(λu.(y (y u))))) (λi.(λo.(λp.p)))) (λa.(λs.a)))
5: (((((λq.(λw.(q (q w)))) (λe.(λr.(λt.(((e (λy.(λu.(u (y r))))) (λi.t)) (λo.o)))))) (λp.(λa.(p (p (p a)))))) (λs.(λd.(λf.f)))) (λg.(λh.g)))
6: ((((λq.((λw.(λe.

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11(), steps_lim=100)

0: (((λq.(λw.((λe.((e (λr.(λt.(λy.y)))) (λu.(λi.u)))) (((λo.(λp.((p (λa.(λs.(λd.(((a (λf.(λg.(g (f s))))) (λh.d)) (λj.j)))))) o))) q) w)))) (λk.(λl.(k (k l))))) (λz.(λx.(z (z (z (z (z x))))))))
1: ((λq.((λw.((w (λe.(λr.(λt.t)))) (λy.(λu.y)))) (((λi.(λo.((o (λp.(λa.(λs.(((p (λd.(λf.(f (d a))))) (λg.s)) (λh.h)))))) i))) (λj.(λk.(j (j k))))) q))) (λl.(λz.(l (l (l (l (l z))))))))
2: ((λq.((q (λw.(λe.(λr.r)))) (λt.(λy.t)))) (((λu.(λi.((i (λo.(λp.(λa.(((o (λs.(λd.(d (s p))))) (λf.a)) (λg.g)))))) u))) (λh.(λj.(h (h j))))) (λk.(λl.(k (k (k (k (k l)))))))))
3: (((((λw.(λe.((e (λr.(λt.(λy.(((r (λu.(λi.(i (u t))))) (λo.y)) (λp.p)))))) w))) (λa.(λs.(a (a s))))) (λd.(λf.(d (d (d (d (d f)))))))) (λg.(λh.(λj.j)))) (λk.(λq.k)))
4: ((((λq.((q (λw.(λe.(λr.(((w (λt.(λy.(y (t e))))) (λu.r)) (λi.i)))))) (λo.(λp.(o (o p)))))) (λa.(λs.(a (a (a (a (a s)))))))) (λd.(λf.(λg.g)))) (λh.(λj.h)))
5: (((((λe.(λr.(e (e (e (e (e r))))))) (λt.(λy.(λu.(((t (λi.(λo.(o (i y))))) (λp.u)) (λa.a)))))) (λs.(λd.(s (s d))))) (λ

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=150)

0: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) e))) (((λi.(λo.((λp.((p (λa.(λs.(λd.d)))) (λf.(λg.f)))) (((λh.(λj.((j (λk.(λl.(λz.(((k (λx.(λc.(c (x l))))) (λv.z)) (λb.b)))))) h))) i) o)))) q) w)) (((λn.(λm.((λq_1.((q_1 (λw_1.(λe_1.(λr_1.r_1)))) (λt_1.(λy_1.t_1)))) (((λu_1.(λi_1.((i_1 (λo_1.(λp_1.(λa_1.(((o_1 (λs_1.(λd_1.(d_1 (s_1 p_1))))) (λf_1.a_1)) (λg_1.g_1)))))) u_1))) n) m)))) w) q)))) (λh_1.(λj_1.(h_1 (h_1 j_1))))) (λk_1.(λl_1.(k_1 (k_1 l_1)))))
1: ((λq.(((λw.(λe.((((λr.(λt.(λy.((r t) y)))) w) e) w))) (((λu.(λi.((λo.((o (λp.(λa.(λs.s)))) (λd.(λf.d)))) (((λg.(λh.((h (λj.(λk.(λl.(((j (λz.(λx.(x (z k))))) (λc.l)) (λv.v)))))) g))) u) i)))) (λb.(λn.(b (b n))))) q)) (((λm.(λq_1.((λw_1.((w_1 (λe_1.(λr_1.(λt_1.t_1)))) (λy_1.(λu_1.y_1)))) (((λi_1.(λo_1.((o_1 (λp_1.(λa_1.(λs_1.(((p_1 (λd_1.(λf_1.(f_1 (d_1 a_1))))) (λg_1.s_1)) (λh_1.h_1)))))) i_1))) m) q_1)))) q) (λj_1.(λk_1.(j_1 (j_1 k_1))))))) (λl_1.(λz_1.(l_1 (l_1 z_1)))))
2: (((λh_1.(λj_1.((((λk_1.(λl_1.(λz_1.((k_1 l_1) z_1)))

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=120)

0: (((λq.(λw.(((λe.(λr.((((λt.(λy.(λu.((t y) u)))) e) r) e))) (((λi.(λo.((λp.((p (λa.(λs.(λd.d)))) (λf.(λg.f)))) (((λh.(λj.((j (λk.(λl.(λz.(((k (λx.(λc.(c (x l))))) (λv.z)) (λb.b)))))) h))) i) o)))) q) w)) (((λn.(λm.((λq_1.((q_1 (λw_1.(λe_1.(λr_1.r_1)))) (λt_1.(λy_1.t_1)))) (((λu_1.(λi_1.((i_1 (λo_1.(λp_1.(λa_1.(((o_1 (λs_1.(λd_1.(d_1 (s_1 p_1))))) (λf_1.a_1)) (λg_1.g_1)))))) u_1))) n) m)))) w) q)))) (λh_1.(λj_1.(h_1 j_1)))) (λk_1.(λl_1.(k_1 (k_1 (k_1 (k_1 l_1)))))))
1: ((λt.(((λy.(λu.((((λi.(λo.(λp.((i o) p)))) y) u) y))) (((λa.(λs.((λd.((d (λf.(λg.(λh.h)))) (λj.(λk.j)))) (((λl.(λz.((z (λx.(λc.(λv.(((x (λb.(λn.(n (b c))))) (λm.v)) (λq_1.q_1)))))) l))) a) s)))) (λw_1.(λe_1.(w_1 e_1)))) t)) (((λr_1.(λt_1.((λy_1.((y_1 (λu_1.(λi_1.(λo_1.o_1)))) (λp_1.(λa_1.p_1)))) (((λs_1.(λd_1.((d_1 (λf_1.(λg_1.(λh_1.(((f_1 (λj_1.(λk_1.(k_1 (j_1 g_1))))) (λl_1.h_1)) (λz_1.z_1)))))) s_1))) r_1) t_1)))) t) (λq.(λw.(q w)))))) (λe.(λr.(e (e (e (e r)))))))
2: (((λq.(λw.((((λe.(λr.(λt.((e r) t)))) q) w) q))) (